In [ ]:
import cv2
import tensorflow as tf
import numpy as np
from classify import Fer,IMAGE_SIZE
from mtcnn.mtcnn import MTCNN

# Initialise the fer model
model = './output'
fer = Fer(model)
fer.build()

# Create face detector
detector = MTCNN()
video = cv2.VideoCapture('./test.mp4')

while(video.isOpened()):
    # Capture frames
    _, frame = video.read()
    # Use MTCNN for detecting faces
    result = detector.detect_faces(frame)
    if result != []:
        for person in result:
            bounding_box = person['box']
            x = bounding_box[0]
            y = bounding_box[1]
            w = bounding_box[0]+bounding_box[2]
            h = bounding_box[1]+bounding_box[3]
            
            face_image = frame[y:h, x:w]
            expression=''
            if face_image is not None and face_image.shape[0] > IMAGE_SIZE and face_image.shape[1] > IMAGE_SIZE:
                face_image = cv2.resize(face_image,(IMAGE_SIZE,IMAGE_SIZE),interpolation = cv2.INTER_AREA)
                face_image = np.array([face_image],dtype=np.float32)
                results = fer.predict(face_image)
                if len(results) > 0:
                    expression = results[0]

            # display resulting frame
            cv2.rectangle(frame,(x,y),(w,h),(0,155,155),2)
            cv2.putText(frame,expression,(x,y),cv2.FONT_HERSHEY_SIMPLEX,1.0,(0,155,155))
            cv2.imshow('frame',frame)
            if cv2.waitKey(1) &0xFF == ord('q'):
                break
video.release()
cv2.destroyAllWindows()        